# 1. Prepare model

In [1]:
import torch
import torch.nn.functional as F

class NaiveModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(1, 20, 5, 1)
        self.conv2 = torch.nn.Conv2d(20, 50, 5, 1)
        self.fc1 = torch.nn.Linear(4 * 4 * 50, 500)
        self.fc2 = torch.nn.Linear(500, 10)
        self.relu1 = torch.nn.ReLU6()
        self.relu2 = torch.nn.ReLU6()
        self.relu3 = torch.nn.ReLU6()
        self.max_pool1 = torch.nn.MaxPool2d(2, 2)
        self.max_pool2 = torch.nn.MaxPool2d(2, 2)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.max_pool1(x)
        x = self.relu2(self.conv2(x))
        x = self.max_pool2(x)
        x = x.view(-1, x.size()[1:].numel())
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [2]:
# define model, optimizer, criterion, data_loader, trainer, evaluator.

import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = NaiveModel().to(device)

optimizer = optim.Adadelta(model.parameters(), lr=1)

criterion = torch.nn.NLLLoss()

transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000)

def trainer(model, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def evaluator(model):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    acc = 100 * correct / len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), acc))

    return acc

In [3]:
# pre-train model for 3 epoches.

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

for epoch in range(0, 3):
    trainer(model, optimizer, criterion, epoch)
    evaluator(model)
    scheduler.step()

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.305085
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.087794
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.071808
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.052543
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.042050
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.059767
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.088583
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.025175
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.069685
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.071096

Test set: Average loss: 0.0387, Accuracy: 9870/10000 (99%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.022943
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.105162
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.015032
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.065901
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.017623
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.014979
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.059367
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.004011
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.075617
T

In [4]:
# show all op_name and op_type in the model.

[print('op_name: {}\nop_type: {}\n'.format(name, type(module))) for name, module in model.named_modules()]

op_name: 
op_type: <class '__main__.NaiveModel'>

op_name: conv1
op_type: <class 'torch.nn.modules.conv.Conv2d'>

op_name: conv2
op_type: <class 'torch.nn.modules.conv.Conv2d'>

op_name: fc1
op_type: <class 'torch.nn.modules.linear.Linear'>

op_name: fc2
op_type: <class 'torch.nn.modules.linear.Linear'>

op_name: relu1
op_type: <class 'torch.nn.modules.activation.ReLU6'>

op_name: relu2
op_type: <class 'torch.nn.modules.activation.ReLU6'>

op_name: relu3
op_type: <class 'torch.nn.modules.activation.ReLU6'>

op_name: max_pool1
op_type: <class 'torch.nn.modules.pooling.MaxPool2d'>

op_name: max_pool2
op_type: <class 'torch.nn.modules.pooling.MaxPool2d'>



[None, None, None, None, None, None, None, None, None, None]

In [5]:
# show the weight size of `conv1`.

print(model.conv1.weight.data.size())

torch.Size([20, 1, 5, 5])


In [6]:
# show the weight of `conv1`.

print(model.conv1.weight.data)

tensor([[[[ 1.4062e-02, -8.6881e-02, -1.5064e-01, -1.4052e-01, -6.2882e-02],
          [-6.7817e-02, -1.1632e-01,  8.8717e-02, -9.9771e-02, -2.4367e-01],
          [ 2.1462e-01,  3.8977e-02, -3.5050e-02, -1.0038e-01, -2.0465e-01],
          [ 5.6569e-02,  1.6690e-02,  1.0047e-03,  1.6719e-01,  6.3830e-02],
          [ 1.0092e-01,  1.6766e-01,  4.7926e-02,  1.4435e-01,  1.8343e-01]]],


        [[[ 6.2452e-02, -1.1161e-01, -2.3515e-01, -1.7988e-01,  8.1673e-02],
          [-1.0257e-01,  1.6148e-01,  1.5648e-02,  1.7896e-01,  7.1744e-02],
          [ 9.2075e-02, -1.1183e-01,  1.6258e-01, -5.3247e-02, -1.8561e-01],
          [-4.7560e-02, -1.5353e-01,  6.0327e-02, -1.4421e-02,  4.3231e-02],
          [ 1.2019e-02,  8.9593e-02,  5.7141e-02, -1.4745e-01,  6.9632e-02]]],


        [[[ 4.5266e-02,  1.3604e-01, -4.2105e-02, -5.8486e-03, -1.9377e-01],
          [ 1.3444e-01, -2.8849e-02,  3.8052e-02,  3.9119e-03, -1.8110e-02],
          [ 1.4361e-01,  1.0363e-01,  1.3143e-01,  6.4049e-02, -1.21

# 2. Prepare config_list for pruning

In [7]:
# we will prune 50% weights in `conv1`.

config_list = [{
    'sparsity': 0.5,
    'op_types': ['Conv2d'],
    'op_names': ['conv1']
}]

# 3. Choose a pruner and pruning

In [8]:
# use l1filter pruner to prune the model

from nni.algorithms.compression.pytorch.pruning import L1FilterPruner

# Note that if you use a compressor that need you to pass a optimizer,
# you need a new optimizer instead of you have used above, because NNI might modify the optimizer.
# And of course this modified optimizer can not be used in finetuning.
pruner = L1FilterPruner(model, config_list)

In [9]:
# we can find the `conv1` has been wrapped, the origin `conv1` changes to `conv1.module`.
# the weight of conv1 will modify by `weight * mask` in `forward()`. The initial mask is a `ones_like(weight)` tensor.

[print('op_name: {}\nop_type: {}\n'.format(name, type(module))) for name, module in model.named_modules()]

op_name: 
op_type: <class '__main__.NaiveModel'>

op_name: conv1
op_type: <class 'nni.compression.pytorch.compressor.PrunerModuleWrapper'>

op_name: conv1.module
op_type: <class 'torch.nn.modules.conv.Conv2d'>

op_name: conv2
op_type: <class 'torch.nn.modules.conv.Conv2d'>

op_name: fc1
op_type: <class 'torch.nn.modules.linear.Linear'>

op_name: fc2
op_type: <class 'torch.nn.modules.linear.Linear'>

op_name: relu1
op_type: <class 'torch.nn.modules.activation.ReLU6'>

op_name: relu2
op_type: <class 'torch.nn.modules.activation.ReLU6'>

op_name: relu3
op_type: <class 'torch.nn.modules.activation.ReLU6'>

op_name: max_pool1
op_type: <class 'torch.nn.modules.pooling.MaxPool2d'>

op_name: max_pool2
op_type: <class 'torch.nn.modules.pooling.MaxPool2d'>



[None, None, None, None, None, None, None, None, None, None, None]

In [10]:
# compress the model, the mask will be updated.

pruner.compress()

NaiveModel(
  (conv1): PrunerModuleWrapper(
    (module): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  )
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
  (relu1): ReLU6()
  (relu2): ReLU6()
  (relu3): ReLU6()
  (max_pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (max_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [11]:
# show the mask size of `conv1`

print(model.conv1.weight_mask.size())

torch.Size([20, 1, 5, 5])


In [12]:
# show the mask of `conv1`

print(model.conv1.weight_mask)

tensor([[[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],


        [[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],


        [[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],


        [[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],


        [[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]],


        [[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]],


        [[[1., 1

In [13]:
# use a dummy input to apply the sparsify.

model(torch.rand(1, 1, 28, 28).to(device))

# the weights of `conv1` have been sparsified.

print(model.conv1.module.weight.data)

tensor([[[[ 0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00],
          [-0.0000e+00, -0.0000e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00],
          [ 0.0000e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]]],


        [[[ 0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,  0.0000e+00],
          [-0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00, -0.0000e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00],
          [-0.0000e+00, -0.0000e+00,  0.0000e+00, -0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -0.0000e+00,  0.0000e+00]]],


        [[[ 0.0000e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00],
          [ 0.0000e+00, -0.0000e+00,  0.0000e+00,  0.0000e+00, -0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -0.00

In [14]:
# export the sparsified model state to './pruned_naive_mnist_l1filter.pth'.
# export the mask to './mask_naive_mnist_l1filter.pth'.

pruner.export_model(model_path='pruned_naive_mnist_l1filter.pth', mask_path='mask_naive_mnist_l1filter.pth')

[2021-07-19 15:44:52] INFO (nni.compression.pytorch.compressor/MainThread) Model state_dict saved to pruned_naive_mnist_l1filter.pth
[2021-07-19 15:44:52] INFO (nni.compression.pytorch.compressor/MainThread) Mask dict saved to mask_naive_mnist_l1filter.pth


# 4. Speed Up

In [15]:
# If you use a wrapped model, don't forget to unwrap it.

pruner._unwrap_model()

# the model has been unwrapped.

print(model)

NaiveModel(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
  (relu1): ReLU6()
  (relu2): ReLU6()
  (relu3): ReLU6()
  (max_pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (max_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [16]:
from nni.compression.pytorch import ModelSpeedup

m_speedup = ModelSpeedup(model, dummy_input=torch.rand(10, 1, 28, 28).to(device), masks_file='mask_naive_mnist_l1filter.pth')
m_speedup.speedup_model()

<ipython-input-1-0f2a9eb92f42>:22: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  x = x.view(-1, x.size()[1:].numel())


[2021-07-19 15:44:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) start to speed up the model
[2021-07-19 15:44:57] INFO (FixMaskConflict/MainThread) {'conv1': 1, 'conv2': 1}


[2021-07-19 15:44:57] INFO (FixMaskConflict/MainThread) dim0 sparsity: 0.500000
[2021-07-19 15:44:57] INFO (FixMaskConflict/MainThread) dim1 sparsity: 0.000000
[2021-07-19 15:44:57] INFO (FixMaskConflict/MainThread) Dectected conv prune dim" 0
[2021-07-19 15:44:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) infer module masks...
[2021-07-19 15:44:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update mask for conv1
[2021-07-19 15:44:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update mask for relu1
[2021-07-19 15:44:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update mask for max_pool1
[2021-07-19 15:44:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update mask for conv2
[2021-07-19 15:44:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update mask for relu2
[2021-07-19 15:44:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update mask for max_pool2
[2021-07-19 1

[2021-07-19 15:44:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update indirect sparsity for relu3
[2021-07-19 15:44:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update the indirect sparsity for the relu3


[2021-07-19 15:44:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update indirect sparsity for fc1
[2021-07-19 15:44:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update the indirect sparsity for the fc1
[2021-07-19 15:44:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update indirect sparsity for .aten::view.9
[2021-07-19 15:44:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update the indirect sparsity for the .aten::view.9
[2021-07-19 15:44:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update indirect sparsity for max_pool2
[2021-07-19 15:44:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update the indirect sparsity for the max_pool2
[2021-07-19 15:44:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update indirect sparsity for relu2
[2021-07-19 15:44:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update the indirect sparsity for the relu2
[202

In [17]:
# the `conv1` has been replace from `Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))` to `Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))`
# and the following layer `conv2` has also changed because the input channel of `conv2` should aware the output channel of `conv1`.

print(model)

NaiveModel(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
  (relu1): ReLU6()
  (relu2): ReLU6()
  (relu3): ReLU6()
  (max_pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (max_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [18]:
# finetune the model to recover the accuracy.

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

for epoch in range(0, 1):
    trainer(model, optimizer, criterion, epoch)
    evaluator(model)

Train Epoch: 0 [0/60000 (0%)]	Loss: 0.349769
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.042060
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.052566
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.011052
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.006365
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.008099
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.024377
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.026260
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.038423
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.014146

Test set: Average loss: 0.0244, Accuracy: 9914/10000 (99%)



# 5. Prepare config_list for quantization

In [19]:
config_list = [{
    'quant_types': ['weight'],
    'quant_bits': {'weight': 8},
    'op_names': ['conv1', 'conv2']
}]

# 6. Choose a quantizer and quantizing

In [20]:
from nni.algorithms.compression.pytorch.quantization import DoReFaQuantizer

quantizer = DoReFaQuantizer(model, config_list)
quantizer.compress()

NaiveModel(
  (conv1): QuantizerModuleWrapper(
    (module): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  )
  (conv2): QuantizerModuleWrapper(
    (module): Conv2d(10, 50, kernel_size=(5, 5), stride=(1, 1))
  )
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
  (relu1): ReLU6()
  (relu2): ReLU6()
  (relu3): ReLU6()
  (max_pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (max_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [21]:
# finetune the model for calibration.

for epoch in range(0, 1):
    trainer(model, optimizer, criterion, epoch)
    evaluator(model)

Train Epoch: 0 [0/60000 (0%)]	Loss: 0.042607
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.135531
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.049159
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.000628
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.010268
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.040661
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.001791
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.075061
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.024104
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.075111

Test set: Average loss: 0.0534, Accuracy: 9897/10000 (99%)



In [22]:
# export the sparsified model state to './quantized_naive_mnist_l1filter.pth'.
# export the calibration config to './calibration_naive_mnist_l1filter.pth'.

quantizer.export_model(model_path='quantized_naive_mnist_l1filter.pth', calibration_path='calibration_naive_mnist_l1filter.pth')

[2021-07-19 15:45:49] INFO (nni.compression.pytorch.compressor/MainThread) Model state_dict saved to quantized_naive_mnist_l1filter.pth
[2021-07-19 15:45:49] INFO (nni.compression.pytorch.compressor/MainThread) Mask dict saved to calibration_naive_mnist_l1filter.pth


{'conv1': {'weight_bit': 8}, 'conv2': {'weight_bit': 8}}

# 7. Speed Up

In [ ]:
# speed up with tensorRT

engine = ModelSpeedupTensorRT(model, (32, 1, 28, 28), config=calibration_config, batchsize=32)
engine.compress()